In [ ]:
from code_analysis import CFG 
from code_analysis import CFGReader
from code_analysis import AST
from code_analysis import ASTReader
import json

cfg = CFGReader().read_cfg('/mnt/c/Users/Othman/Desktop/TP-LOG6302/log6302a_lab5/part_1/file_4.php.cfg.json')
ast = ASTReader().read_ast('/mnt/c/Users/Othman/Desktop/TP-LOG6302/log6302a_lab5/part_1/file_4.php.ast.json')


def referencesExpression (cfg : CFG, node, ref):
    
    if cfg.get_type(node) == 'BinOP':
        referencesExpression(cfg, cfg.get_op_hands(node)[0], ref)
        referencesExpression(cfg, cfg.get_op_hands(node)[1], ref)
    else : 
        ref.append(node)
    return ref

def poss_tainted_defs(cfg : CFG ) :

    taint_file = open('/mnt/c/Users/Othman/Desktop/TP-LOG6302/log6302a_lab5/part_1/file_4.php.taint.json')
    data = json.load(taint_file)
    defs = data['defs']
    refs = data['refs']
    pairs = data['pairs']
    sinks = data['sinks']
    filters = data['filters']
    safes = data['safes']
    sources = data['sources']


    nodeSet = cfg.get_node_ids()

    IN = {}
    OUT = {}
    tainted_KILL = {}
    tainted_GEN = {}

    for node in nodeSet :
        
        IN[node] = set()
        OUT[node] = set()
        tainted_GEN[node] = set()
        tainted_KILL[node] = set()
       
    old_OUT = {}

    changes = True
    
    while changes :

        changes = False
        
        for node in nodeSet :

            if cfg.get_type(node) == 'BinOP' and cfg.get_image(node) == '=' :
                
                left, right = cfg.get_op_hands(node)
                
                if right in sources :
                    tainted_GEN[node].add(left)
                elif right in safes :
                    tainted_GEN[node] = set()
                elif right in filters :
                    tainted_GEN[node] = set()
                else : 
                    refExpr = referencesExpression (cfg , right, [])
                    for ref in refExpr :
                        for defRef in pairs :
                            definition , reference = defRef
                            if reference == ref and definition in IN[node]:    
                                tainted_GEN[node].add(left)
                                break
                          
                for definition in defs : 
                    if definition == left :
                        tainted_KILL[node].add(definition) 
            
            
            predNodes = []
            if cfg.get_type(node) == 'CallEnd':
                predNodes.append(cfg.get_call_begin(node))
            else :
                predNodes = cfg.get_parents(node)
            
            for predNode in predNodes :
                IN[node] = IN[node].union(OUT[predNode])

            old_OUT[node] = OUT [node]
            OUT[node] = tainted_GEN[node].union(IN[node].difference(tainted_KILL[node]))
            if OUT[node] !=  old_OUT[node]:
                changes = True

    for sink in sinks : 
        for pair in pairs :
            definition, reference = pair
            if sink == reference and definition in IN[sink]:
                print(pair)
                #print('end')
        
    return IN


print(poss_tainted_defs(cfg))
            
